# Samhliðamálheild fyrir ensku og íslensku - ParIce
Hér er unnið með ParIce málheildina. Gangasafnið lesið, prófunargögn síuð út og sett í staðlað form.

Gert er ráð fyrir að eftirfarandi gögn séu til staðar
- ParIce eins og það er sótt á málföng, `parice_dir`
- Prófunargögn eins og fengin frá Steinþóri, `test_dir`

Eftir að hafa keyrt reikniritið verður `target_dir` eftirfarandi
- `target_dir/x.(is|en)`, þar sem `x` er undirmálheild úr ParIce.
- `target_dir/train.(is|en)` allar málheildir úr ParIce sameinaðar og prófunar- og fínpússunargögn fjarlægð.
- `target_dir/dev.(is|en)` 2000 setningapör valin af handahófi úr sameinuðu ParIce með prófunargögn fjarlægð.
- `target_dir/test-(ees|ema|opensubtitles).(is|en)` Prófunargögn eins og fengin frá Steinþóri. 

Hægt er að lesa ParIce gögnin á tvo vegu, í gegnum `.xml` eða `.tmx`. `.xml` skjölin eru tilreidd, mörkuð og lemmuð. Undanskilið eru opensubtitles undirmálheildin. `.tmx` skjölin innihalda paraðar setningar. Það er, `.xml` er yfirmengi `.tmx`.

**Ath** Við lestur á `.xml`-skjölum tókum við eftir því að gamall Tokenizer hafði verið notaður til þess að tilreiða og breytti enskum lokunar-gæsalöppum " í íslenskar lokunar-gæsalappir “. Út af þessari villu var ákveðið að lesa `.tmx`-skjölin og svo marka þau og lemma sérstaklega síðar.

Reikniritinu er skipt í eftirfarandi skref:
1. Uppsetning
2. Sækja opensubtitles og búa til `.tmx` skrá úr þeim.
3. Lesa málheildina, sem annað hvort `.xml` eða `.tmx`.
  1. Sem `.xml`
  2. Sem `.tmx`
4. Sía út prófunargögn
5. Skipta í dev/train
6. Skrifa gögn og taka til

## 1. Uppsetning

In [1]:
# The location of Parice to read
parice_dir = '/work/haukurpj/data/raw/Parice1.0'
# The location of the Parice test data (zipped, provided separatly).
test_set_zip = '/work/haukurpj/data/raw/prófunarsett-20191205T064220Z-001.zip'

# The location of where to write the results
target_dir = '/work/haukurpj/data/formatted/Parice1.0'

from glob import glob
from pprint import pprint

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pathlib

parice_raw_dir = pathlib.Path(parice_dir)
parice_formatted_dir = pathlib.Path(target_dir)
parice_test_zip = pathlib.Path(test_set_zip)

In [3]:
assert parice_raw_dir.exists()
assert parice_test_zip.exists()
if not parice_formatted_dir.exists():
    parice_formatted_dir.mkdir()

## 2. Sækja opensubtitles
Opensubtitles er ekki deilt með ParIce, út af leyfismálum, svo við þurfum að sækja þau gögn og keyra scriptu til þess að para setningar.

In [99]:
!wget -P {parice_raw_dir}/opensubtitles/en.zip http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/raw/en.zip
!wget -P {parice_raw_dir}/opensubtitles/is.zip http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/raw/is.zip

--2020-02-05 15:48:02--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/raw/is.zip
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/raw/is.zip [following]
--2020-02-05 15:48:02--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/raw/is.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60620921 (58M) [application/zip]
Saving to: ‘/work/haukurpj/data/raw/Parice1.0/opensubtitles/is.zip/download.php?f=OpenSubtitles%2Fv2018%2Fraw%2Fis.zip.1’

download.php?f=Open 100%[===================>]  57.81M  4.93MB/s    in 14s     

2020-02-05 15:48:17 (4.17 MB/s) - ‘/work/haukurpj/data/raw/Parice1.0/opensubtitles/is.zip/downl

In [94]:
!mkdir -p {parice_raw_dir}/opensubtitles/en
!mkdir -p {parice_raw_dir}/opensubtitles/is

In [97]:
!ls {parice_raw_dir}/opensubtitles/

README	en  en.zip  is	is.zip	links_to_opus  read_opus.py


In [ ]:
!unzip {parice_raw_dir}/opensubtitles/en.zip/*.zip -d {parice_raw_dir}/opensubtitles/en
!unzip {parice_raw_dir}/opensubtitles/is.zip/*.zip -d {parice_raw_dir}/opensubtitles/is

### Keyra scriptu til þess að búa til opensubtitles.tmx
Núna þarf að keyra scriptu sem býr til samhliða gögnin fyrir opensubtitles. Til þess þurfum við vísa í íslensku og ensku gögnin og scriptan finnur sjálf `links_to_opus`.

In [ ]:
%cd {parice_raw_dir}/opensubtitles
!python read_opus.py --input {parice_raw_dir}/opensubtitles --output {parice_raw_dir}/tmx

In [101]:
!ls -lh {parice_raw_dir}/tmx

total 1.1G
-rwxrwxrwx 1 haukurpj local-staff 4.7M Mar 25  2019 baekur.tmx
-rwxrwxrwx 1 haukurpj local-staff  16M Mar 25  2019 biblian.tmx
-rwxrwxrwx 1 haukurpj local-staff 583M Sep 19 11:31 ees.tmx
-rwxrwxrwx 1 haukurpj local-staff 122M Mar 25  2019 ema.tmx
-rwxrwxrwx 1 haukurpj local-staff 5.3M Jun 11  2019 eso.tmx
-rwxrwxrwx 1 haukurpj local-staff 5.9M Mar 25  2019 fornritin.tmx
-rwxrwxrwx 1 haukurpj local-staff 1.2M Jun 11  2019 hagstofan.tmx
-rwxrwxrwx 1 haukurpj local-staff 8.7M Jun 11  2019 kde4.tmx
-rw-r--r-- 1 haukurpj local-staff 299M Feb  5 15:50 opensubtitles.tmx
-rwxrwxrwx 1 haukurpj local-staff 1.5M Mar 25  2019 tatoeba.tmx
-rwxrwxrwx 1 haukurpj local-staff 1.7M Mar 25  2019 ubuntu.tmx


In [102]:
!wc -l {parice_raw_dir}/tmx/*.tmx

     62087 /work/haukurpj/data/raw/Parice1.0/tmx/baekur.tmx
    260972 /work/haukurpj/data/raw/Parice1.0/tmx/biblian.tmx
   8505867 /work/haukurpj/data/raw/Parice1.0/tmx/ees.tmx
   1617339 /work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx
     63172 /work/haukurpj/data/raw/Parice1.0/tmx/eso.tmx
     87992 /work/haukurpj/data/raw/Parice1.0/tmx/fornritin.tmx
     11448 /work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx
    199643 /work/haukurpj/data/raw/Parice1.0/tmx/kde4.tmx
   6523036 /work/haukurpj/data/raw/Parice1.0/tmx/opensubtitles.tmx
     33059 /work/haukurpj/data/raw/Parice1.0/tmx/tatoeba.tmx
     42295 /work/haukurpj/data/raw/Parice1.0/tmx/ubuntu.tmx
  17406910 total


## 3. Lesa ParIce

Við skilgreinum gagnskipan, til þess að auka læsileika. Þessi kóði er notaður fyrir `.xml` og `.tmx`

In [4]:
from typing import List, Tuple
Sentence = List
POS = List
Lemma = List

Corpus = List[Sentence]

EnrichedSentence = Tuple[Sentence, POS, Lemma]
EnrichedCorpus = List[EnrichedSentence]

In [5]:
import tokenizer

def path_to_corpus(in_path: str) -> Corpus:
    with open(in_path) as f_in:
        return f_in.readlines()
    
def corpus_to_enriched_corpus(corpus: Corpus) -> EnrichedCorpus:
    enriched_corpus: EnrichedCorpus = []
    for sentence in corpus:
        enriched_corpus.append(([token for tok in tokenizer.tokenize(sentence) if tok.txt is not None for token in tok.txt.split(' ')],
                               list(),
                               list()))
    return enriched_corpus

### 3.1 Sem `.xml` skrár
Í stað þess að lesa tmx skrárnar er hægt að lesa xml skrárnar, þar eru setningar tilreiddar, markaðar og lemmaðar. Við þurfum þó tilreiða, marka og lemma OpenSubtitiles gögnin. Við gerum það síðar og notum API frá SÁM til þess.

In [4]:
def list_documents(directory, lang):
    return glob(f'{directory}/xml/*/{lang}/*.xml')

In [5]:
en_docs = list_documents(parice_raw_dir, 'en')
is_docs = list_documents(parice_raw_dir, 'is')

In [6]:
print(len(en_docs), len(is_docs))
print(en_docs[0])

8550 8550
/work/haukurpj/data/raw/Parice1.0/xml/ees/en/32012r0600.xml


In [8]:
from xml.etree import ElementTree as ET

def read_enriched_corpora(docs: List[str]) -> EnrichedCorpus:
    lines: EnrichedCorpus = []
    for doc in docs:
        lines += read_enriched_corpus(doc)
    return lines

ALLOWED_TAGS = set(('{http://www.tei-c.org/ns/1.0}w', '{http://www.tei-c.org/ns/1.0}c'))
def read_enriched_corpus(doc: str) -> EnrichedCorpus:
    NS = {'a': 'http://www.tei-c.org/ns/1.0'}
    root = ET.parse(doc).getroot()
    lines: EnrichedCorpus = []
    # We gather all the segments from the body, avoiding the divs
    for seg in root.findall('.//a:body//a:seg', NS):
        form: Sentence = list()
        pos: POS = list()
        lemma: Lemma = list()
        line: EnrichedSentence = (form, pos, lemma)
        for s in seg.findall('.//a:s', NS):
            for node in s.getchildren():
                # print(node.text, node.tag, s.text, seg.attrib['id'], doc)
                assert node.tag in ALLOWED_TAGS
                form.append(node.text)
                pos.append(node.attrib['type'])
                lemma.append(node.attrib['lemma'])                    
        lines.append(line)
    return lines

In [9]:
enriched_corpus_en = read_enriched_corpora(en_docs)
enriched_corpus_is = read_enriched_corpora(is_docs)

/home/staff/haukurpj/.conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


Við ættum að fá 2284424 línur fyrir hvora málheildina.

In [10]:
print(len(enriched_corpus_en), len(enriched_corpus_is))
print(enriched_corpus_en[10], enriched_corpus_is[10])

2284424 2284424
(['•', 'Regulation', '(', 'EC', ')', 'No', 'of', 'the', 'European', 'Parliament', 'and', 'of', 'the', 'Council', 'of', '25', 'November', '2009', 'on', 'the', 'voluntary', 'participation', 'by', 'organisations', 'in', 'a', 'Community', 'eco-management', 'and', 'audit', 'scheme', '(', 'EMAS', ')', ',', 'repealing', 'Regulation', '(', 'EC', ')', 'No', 'and', 'Commission', 'Decisions', '2001/681/EC', 'and', '2006/193/EC', '(', '1', ')', 'provides', 'for', 'an', 'independent', 'and', 'neutral', 'accreditation', 'or', 'licensing', 'system', 'for', 'environmental', 'verifiers', '.'], ['JJ', 'NNP', '(', 'NNP', ')', 'NNP', 'IN', 'DT', 'NNP', 'NNP', 'CC', 'IN', 'DT', 'NNP', 'IN', 'CD', 'NNP', 'CD', 'IN', 'DT', 'JJ', 'NN', 'IN', 'NNS', 'IN', 'DT', 'NNP', 'NN', 'CC', 'NN', 'NN', '(', 'NNP', ')', ',', 'VBG', 'NNP', '(', 'NNP', ')', 'NNP', 'CC', 'NNP', 'NNP', 'CD', 'CC', 'CD', '(', 'CD', ')', 'VBZ', 'IN', 'DT', 'JJ', 'CC', 'JJ', 'NN', 'CC', 'NN', 'NN', 'IN', 'JJ', 'NNS', '.'], ['•', 

Núna lesum við opensubtitles tmx skrána.

**ATH** að `.tmx` skráin er ekki tilreidd, og því verðum við að tilreiða hana svo hún líti eins út og hinar skrárnar. Enn fremur þá gerir tilreiðslan okkur kleift að gera athuganir síðar meir í kóðanum.

In [11]:
from frontend import bulk as b
opensub_en, opensub_is = b.tmx_split([parice_raw_dir.joinpath('tmx').joinpath('opensubtitles.tmx')], 'EN-GB', 'IS-IS')[0]

2020-02-06 11:18:08,439 - frontend - INFO - Initialized


In [12]:
opensub_en, opensub_is

(PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/opensubtitles.en'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/opensubtitles.is'))

In [14]:
enriched_corpus_is.extend(corpus_to_enriched_corpus(path_to_corpus(opensub_is)))
enriched_corpus_en.extend(corpus_to_enriched_corpus(path_to_corpus(opensub_en)))

In [15]:
len(enriched_corpus_en)
len(enriched_corpus_is)
enriched_corpus_en[-10], enriched_corpus_is[-10]

3589030

3589030

((['I', 'understand', 'now', '.'], [], []),
 (['Ég', 'skil', 'þetta', 'núna', '.'], [], []))

Fjöldi lína (3589030) passar við fjölda lína við `.tmx` lestur.

In [75]:
test

[(['GEN.1.1'], ['NN'], ['GEN.1.1']),
 (['In',
   'the',
   'beginning',
   'God',
   'created',
   'the',
   'heaven',
   'and',
   'the',
   'earth'],
  ['IN', 'DT', 'NN', 'NNP', 'VBD', 'DT', 'NN', 'CC', 'DT', 'NN'],
  ['In',
   'the',
   'beginning',
   'God',
   'create',
   'the',
   'heaven',
   'and',
   'the',
   'earth']),
 (['GEN.1.2'], ['NN'], ['GEN.1.2']),
 (['And',
   'the',
   'earth',
   'was',
   'without',
   'form',
   'and',
   'void',
   'and',
   'darkness',
   'was',
   'upon',
   'the',
   'face',
   'of',
   'the',
   'deep'],
  ['CC',
   'DT',
   'NN',
   'VBD',
   'IN',
   'NN',
   'CC',
   'NN',
   'CC',
   'NN',
   'VBD',
   'IN',
   'DT',
   'NN',
   'IN',
   'DT',
   'NN'],
  ['And',
   'the',
   'earth',
   'be',
   'without',
   'form',
   'and',
   'void',
   'and',
   'darkness',
   'be',
   'upon',
   'the',
   'face',
   'of',
   'the',
   'deep']),
 (['And',
   'the',
   'Spirit',
   'of',
   'God',
   'moved',
   'upon',
   'the',
   'face',
   'of'

### 3.2 Sem `.tmx` skrár
Við getum líka lesið tmx skrárnar, en þær innihalda ekki eins miklar upplýsingar og xml skrárnar.
Fyrst þarf þó aðeins að lagfæra skrárnar svo þær séu allar með sama tungumála merki.

In [47]:
!grep -m 2 "lang=\"is\"" {parice_raw_dir}/tmx/*.tmx
!grep -m 2 "lang=\"en\"" {parice_raw_dir}/tmx/*.tmx

/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx:  <header adminlang="EN-US" creationdate="20170426T083707Z" creationtool="Moses-to-TMX-converer" creationtoolversion="1.0" datatype="plaintext" o-tmf="Moses plain text files" segtype="sentence" srclang="en">
/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx:  <header adminlang="en" creationdate="20170319T213035Z" creationtool="mALIGNa" creationtoolversion="2" datatype="plaintext" o-tmf="al" segtype="block" srclang="en">


In [48]:
!sed -i 's/xml:lang="is"/xml:lang="IS-IS"/g' {parice_raw_dir}/tmx/hagstofan.tmx
!sed -i 's/xml:lang="en"/xml:lang="EN-GB"/g' {parice_raw_dir}/tmx/hagstofan.tmx
!sed -i 's/xml:lang="is"/xml:lang="IS-IS"/g' {parice_raw_dir}/tmx/ema.tmx
!sed -i 's/xml:lang="en"/xml:lang="EN-GB"/g' {parice_raw_dir}/tmx/ema.tmx

In [49]:
!grep -m 2 "lang=\"is\"" {parice_raw_dir}/tmx/*.tmx
!grep -m 2 "lang=\"en\"" {parice_raw_dir}/tmx/*.tmx

/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx:  <header adminlang="EN-US" creationdate="20170426T083707Z" creationtool="Moses-to-TMX-converer" creationtoolversion="1.0" datatype="plaintext" o-tmf="Moses plain text files" segtype="sentence" srclang="en">
/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx:  <header adminlang="en" creationdate="20170319T213035Z" creationtool="mALIGNa" creationtoolversion="2" datatype="plaintext" o-tmf="al" segtype="block" srclang="en">


In [6]:
tmx_files = glob(f'{parice_raw_dir}/tmx/*.tmx')
tmx_files = [pathlib.Path(tmx_file) for tmx_file in tmx_files]
pprint(tmx_files)

[PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/ubuntu.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/tatoeba.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/eso.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/biblian.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/fornritin.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/ees.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/baekur.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/kde4.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/opensubtitles.tmx')]


In [51]:
import frontend.bulk as b

parice_tmx = b.tmx_split(tmx_files, 'EN-GB', 'IS-IS')
en_parice, is_parice = zip(*parice_tmx)

In [50]:
for corpus in en_parice + is_parice:
    !mv {corpus} {parice_formatted_dir}

NameError: name 'en_parice' is not defined

In [7]:
corpora = [corpus.stem for corpus in tmx_files]
corpora.sort()
corpora

['baekur',
 'biblian',
 'ees',
 'ema',
 'eso',
 'fornritin',
 'hagstofan',
 'kde4',
 'opensubtitles',
 'tatoeba',
 'ubuntu']

In [8]:
parice_en_list = [f'{parice_formatted_dir}/{corpus}.en' for corpus in corpora]
parice_is_list = [f'{parice_formatted_dir}/{corpus}.is' for corpus in corpora]
pprint(parice_en_list)
pprint(parice_is_list)

['/work/haukurpj/data/formatted/Parice1.0/baekur.en',
 '/work/haukurpj/data/formatted/Parice1.0/biblian.en',
 '/work/haukurpj/data/formatted/Parice1.0/ees.en',
 '/work/haukurpj/data/formatted/Parice1.0/ema.en',
 '/work/haukurpj/data/formatted/Parice1.0/eso.en',
 '/work/haukurpj/data/formatted/Parice1.0/fornritin.en',
 '/work/haukurpj/data/formatted/Parice1.0/hagstofan.en',
 '/work/haukurpj/data/formatted/Parice1.0/kde4.en',
 '/work/haukurpj/data/formatted/Parice1.0/opensubtitles.en',
 '/work/haukurpj/data/formatted/Parice1.0/tatoeba.en',
 '/work/haukurpj/data/formatted/Parice1.0/ubuntu.en']
['/work/haukurpj/data/formatted/Parice1.0/baekur.is',
 '/work/haukurpj/data/formatted/Parice1.0/biblian.is',
 '/work/haukurpj/data/formatted/Parice1.0/ees.is',
 '/work/haukurpj/data/formatted/Parice1.0/ema.is',
 '/work/haukurpj/data/formatted/Parice1.0/eso.is',
 '/work/haukurpj/data/formatted/Parice1.0/fornritin.is',
 '/work/haukurpj/data/formatted/Parice1.0/hagstofan.is',
 '/work/haukurpj/data/form

In [9]:
import operator
from functools import reduce

enriched_corpus_is = reduce(operator.iadd, map(corpus_to_enriched_corpus, map(path_to_corpus, parice_is_list)), list())
enriched_corpus_en = reduce(operator.iadd, map(corpus_to_enriched_corpus, map(path_to_corpus, parice_en_list)), list())

In [10]:
len(enriched_corpus_is), len(enriched_corpus_en)
enriched_corpus_is[:2], enriched_corpus_en[:2]

(3589030, 3589030)

([(['FYRRI', 'ÞÁTTUR'], [], []),
  (['I.', 'Veizlan', 'í', 'nýju', 'höllinni', '.'], [], [])],
 [(['BOOK', 'I'], [], []),
  (['CHAPTER', 'I', 'A', 'FETE', 'AT', 'THE', 'NEW', 'PALACE'], [], [])])

In [ ]:
import pickle

with open(parice_formatted_dir.joinpath('enriched.pkl', 'wb') as f_out:
    pickle.dump(enriched_corpus_lemma_is, f_out)

## 4. Sía út prófunargögn
Núna síum við út prófunargögnin úr ParIce. Byrjum á því að afpakka prófunargögn

In [33]:
!unzip {parice_test_zip} -d {parice_formatted_dir}

Archive:  /work/haukurpj/data/raw/prófunarsett-20191205T064220Z-001.zip
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/LESTU-MIG.docx  
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/ees.csv  
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/opensubtitles.csv  
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/ema.csv  


In [34]:
!ls {parice_formatted_dir}/prófunarsett

LESTU-MIG.docx	ees.csv  ema.csv  opensubtitles.csv


In [35]:
def read_csv(p_in):
    en_text = []
    is_text = []
    with p_in.open() as f_in:
        for index, line in enumerate(f_in):
            if '### SKJAL ID' in line:
                continue
            if line == '\n':
                continue
            id, en_sent, is_sent = line.split('\t')
            en_text.append(en_sent+'\n')
            is_text.append(is_sent)
    return en_text, is_text

In [36]:
ees_en, ees_is = read_csv(parice_formatted_dir.joinpath('prófunarsett').joinpath('ees.csv'))
ema_en, ema_is = read_csv(parice_formatted_dir.joinpath('prófunarsett').joinpath('ema.csv'))
opensubtitles_en, opensubtitles_is = read_csv(parice_formatted_dir.joinpath('prófunarsett').joinpath('opensubtitles.csv'))
parice_test_is = ees_is + ema_is + opensubtitles_is
parice_test_en = ees_en + ema_en + opensubtitles_en

In [ ]:
def get_indices(corpus, sentences):
    indices = []
    for index, line in enumerate(corpus):
        if line in sentences:
            indices.append(index)
    return indices

In [ ]:
indicies_test_en = get_indices(parice_en, frozenset(parice_test_en))
indicies_test_is = get_indices(parice_is, frozenset(parice_test_is))
print(len(indicies_test_en), len(indicies_test_is))

Núna þurfum við að fjarlægja setningar úr ParIce sem eru í prófunargögnunum.

In [ ]:
indicies_test_intersection = set(indicies_test_is).intersection(set(indicies_test_en))
indicies_test_union = set(indicies_test_is).union(set(indicies_test_en))
print(len(indicies_test_intersection))
print(len(indicies_test_union))
print(len(parice_test_en), len(parice_test_is))

Við notum sammengið, til þess að losa okkur við allan efa um að þýðingarvélin hafi séð þessar setningar áður og einnig til þess að vera með sambærilega vinnslu og Miðeind.

In [ ]:
def drop_indices(corpus, indices):
    filtered_corpus = []
    indices = list(indices)
    indices.sort()
    current_index = 0
    for corpus_index, line in enumerate(corpus):
        if current_index == len(indices):
            filtered_corpus.append(line)
        elif indices[current_index] == corpus_index:
            current_index += 1
        else:
            filtered_corpus.append(line)
    return filtered_corpus

In [ ]:
drop = indicies_test_union
parice_no_test_en = drop_indices(parice_en, drop)
parice_no_test_is = drop_indices(parice_is, drop)

## 5. Skipta í dev/train
Við skiptum restini af gögnunum í tvennt, dev/train. Við veljum 2000 setningapör í dev og fjarlægjum þær úr restinni. Sá afgangur verður train. Það er hugsanlegt að við veljum tvítekningar í dev en við verðum bara að lifa með það.

In [ ]:
seed = 10

import random
random.seed(seed)
parice_dev_en = list(random.sample(parice_no_test_en, 2000))
indieces_dev_en = get_indices(parice_no_test_en, frozenset(parice_dev_en))
random.seed(seed)
parice_dev_is = list(random.sample(parice_no_test_is, 2000))
indieces_dev_is = get_indices(parice_no_test_is, frozenset(parice_dev_is))

In [ ]:
print(len(indieces_dev_en), len(indieces_dev_is))

Hægt er að keyra kóðann (illa skrifaður) fyrir neðan til þess að athuga hvort niðurstöðurnar eru réttar. Þær virðast vera það.

In [ ]:
#for line in parice_dev_en:
#    print(line)
#    for index, p_line in enumerate(parice_no_test_en):
#        if p_line == line:
#            print(index, p_line)
#            if index not in indieces_dev_en:
#                raise RuntimeError("Error! Not found")

In [ ]:
indicies_dev_intersection = set(indieces_dev_en).intersection(set(indieces_dev_is))
indicies_dev_union = set(indieces_dev_en).union(set(indieces_dev_is))
print(len(indicies_dev_intersection))
print(len(indicies_dev_union))
print(len(parice_dev_en), len(parice_dev_is))

In [ ]:
for i in range(2):
    print(parice_dev_en[i], parice_dev_is[i])

In [ ]:
drop = indicies_dev_union
parice_train_en = drop_indices(parice_no_test_en, drop)
parice_train_is = drop_indices(parice_no_test_is, drop)

Athugum hvort línufjöldinn gangi ekki upp.

In [ ]:
print(len(parice_train_en), len(parice_train_is))
print(len(parice_en) - (len(parice_train_en) + len(indicies_dev_union) + len(indicies_test_union)))

## 6. Skrifa gögn og hreinsa til
Núna skrifum við gögnin niður í skrár og fjarlægjum tímabundnar skrár.

In [ ]:
!ls {target_dir}

In [ ]:
def write_to_file(corpus, path):
    with path.open('w+') as f_in:
        for line in corpus:
            f_in.write(line)
    !wc -l {path}

In [ ]:
write_to_file(parice_train_en, parice_formatted_dir.joinpath('train.en'))
write_to_file(parice_train_is, parice_formatted_dir.joinpath('train.is'))
write_to_file(parice_dev_en, parice_formatted_dir.joinpath('dev.en'))
write_to_file(parice_dev_is, parice_formatted_dir.joinpath('dev.is'))
write_to_file(ees_en, parice_formatted_dir.joinpath('test-ees.en'))
write_to_file(ees_is, parice_formatted_dir.joinpath('test-ees.is'))
write_to_file(ema_en, parice_formatted_dir.joinpath('test-ema.en'))
write_to_file(ema_is, parice_formatted_dir.joinpath('test-ema.is'))
write_to_file(opensubtitles_en, parice_formatted_dir.joinpath('test-opensubtitles.en'))
write_to_file(opensubtitles_is, parice_formatted_dir.joinpath('test-opensubtitles.is'))

In [ ]:
!rm -rf {parice_formatted_dir}/prófunarsett

In [ ]:
!ls {target_dir}